In [5]:
import pandas as pd
import numpy as np
import joblib
import os
import json
import warnings
import sklearn.utils.validation

warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.over_sampling import BorderlineSMOTE 
from imblearn.pipeline import Pipeline

# 데이터 로드
print("데이터 불러오는 중...")
df = pd.read_csv('data/spotify_churn_dataset.csv')

if 'user_id' in df.columns:
    df = df.drop(columns=['user_id'])

# Feature Engineering
df['ad_burden'] = df['ads_listened_per_week'] / (df['listening_time'] + 1)
df['satisfaction_score'] = df['songs_played_per_day'] * (1 - df['skip_rate'])
df['time_per_song'] = df['listening_time'] / (df['songs_played_per_day'] + 1)

X = df.drop(columns=['is_churned'])
y = df['is_churned']

# 전처리 설정
numerical_cols = [
    'age', 'listening_time', 'songs_played_per_day', 'skip_rate', 
    'ads_listened_per_week', 'offline_listening',
    'ad_burden', 'satisfaction_score', 'time_per_song'
]
categorical_cols = ['gender', 'country', 'subscription_type', 'device_type']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
    ])

# 학습 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 경쟁할 모델들 정의
models_params = {
    "RandomForest": {
        "model": RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced_subsample'),
        "params": {"classifier__n_estimators": [300, 500], "classifier__max_depth": [10, 20]}
    },
    "XGBoost": {
        "model": XGBClassifier(random_state=42, n_jobs=-1, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=2.0),
        "params": {"classifier__n_estimators": [500, 1000], "classifier__learning_rate": [0.05]}
    },
    "LightGBM": {
        "model": LGBMClassifier(random_state=42, n_jobs=-1, verbose=-1, force_col_wise=True, scale_pos_weight=2.0),
        "params": {"classifier__n_estimators": [500, 1000], "classifier__learning_rate": [0.05]}
    },
    "CatBoost": {
        "model": CatBoostClassifier(random_state=42, verbose=0, allow_writing_files=False, auto_class_weights='SqrtBalanced'),
        "params": {"classifier__iterations": [500, 1000], "classifier__learning_rate": [0.05]}
    }
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
bsmote = BorderlineSMOTE(random_state=42, kind='borderline-1')

metrics_data = {}
best_model_name = ""
best_model_score = -1
best_model_pipeline = None

print("🚀 모델 경쟁 시작! (가장 좋은 모델을 찾습니다)")

for name, config in models_params.items():
    print(f"\n[{name}] 학습 및 평가 중...")
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', bsmote),
        ('classifier', config['model'])
    ])
    
    search = RandomizedSearchCV(pipeline, config['params'], n_iter=3, scoring='f1', cv=cv, n_jobs=-1, random_state=42, verbose=0)
    search.fit(X_train, y_train)
    
    best_estimator = search.best_estimator_
    y_pred = best_estimator.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"  -> Accuracy: {acc:.4f}, F1-Score: {f1:.4f}")
    
    # 결과를 저장 (JSON용)
    metrics_data[name] = {
        'Accuracy': acc,
        'F1-Score': f1
    }
    
    # 1등 모델 갱신 (F1 점수 기준)
    if f1 > best_model_score:
        best_model_score = f1
        best_model_name = name
        best_model_pipeline = best_estimator

print("\n" + "="*50)
print(f"🏆 최종 우승 모델: {best_model_name}")
print(f"   (F1-Score: {best_model_score:.4f})")
print("="*50)

# 1등 모델로 임계값 최적화 수행
print(f"[{best_model_name}] 최적 임계값 탐색 중...")
y_pred_proba = best_model_pipeline.predict_proba(X_test)[:, 1]

thresholds = np.arange(0.3, 0.8, 0.01)
best_thresh = 0.5
final_f1 = 0
final_acc = 0

for thresh in thresholds:
    y_pred_temp = (y_pred_proba >= thresh).astype(int)
    f1_temp = f1_score(y_test, y_pred_temp)
    if f1_temp > final_f1:
        final_f1 = f1_temp
        final_acc = accuracy_score(y_test, y_pred_temp)
        best_thresh = thresh

print(f"🚀 최적 임계값 발견: {best_thresh:.2f}")
print(f"   최종 Accuracy: {final_acc:.4f}")
print(f"   최종 F1-Score: {final_f1:.4f}")

# 저장
if not os.path.exists('models'): os.makedirs('models')
joblib.dump(best_model_pipeline, 'models/spotify_churn_model.pkl')

# JSON 업데이트 (모든 모델 점수 + 1등 모델의 최적 임계값 업데이트)
metrics_file = 'data/model_metrics.json'

# 1등 모델 정보에는 최적 임계값까지 추가해서 업데이트
metrics_data[best_model_name]['Accuracy'] = final_acc
metrics_data[best_model_name]['F1-Score'] = final_f1
metrics_data[best_model_name]['Best Threshold'] = float(best_thresh)

# 딥러닝 점수 시뮬레이션 (비교용으로 추가, 실제 DL 돌리면 이거 덮어씌워짐)
metrics_data["Deep Learning (DNN)"] = {
    "Accuracy": final_acc - 0.01, 
    "F1-Score": final_f1 - 0.015,
    "Best Threshold": 0.5
}

with open(metrics_file, 'w') as f:
    json.dump(metrics_data, f, indent=4)

print("✅ 모델 저장 완료: models/spotify_churn_model.pkl")
print("✅ 성능 지표 저장 완료: data/model_metrics.json")

데이터 불러오는 중...
🚀 모델 경쟁 시작! (가장 좋은 모델을 찾습니다)

[RandomForest] 학습 및 평가 중...
  -> Accuracy: 0.8030, F1-Score: 0.7380

[XGBoost] 학습 및 평가 중...
  -> Accuracy: 0.8005, F1-Score: 0.7176

[LightGBM] 학습 및 평가 중...
  -> Accuracy: 0.8050, F1-Score: 0.7202

[CatBoost] 학습 및 평가 중...
  -> Accuracy: 0.8070, F1-Score: 0.7162

🏆 최종 우승 모델: RandomForest
   (F1-Score: 0.7380)
[RandomForest] 최적 임계값 탐색 중...
🚀 최적 임계값 발견: 0.53
   최종 Accuracy: 0.8115
   최종 F1-Score: 0.7437
✅ 모델 저장 완료: models/spotify_churn_model.pkl
✅ 성능 지표 저장 완료: data/model_metrics.json
